In [1]:
!pip install pytorch_metric_learning;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 KB 570.7 kB/s eta 0:00:00


In [2]:
import os
import random
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
from tqdm.auto import tqdm
import albumentations as A

from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from pytorch_metric_learning import losses

In [4]:
from transformers import ViTFeatureExtractor, ViTModel

In [5]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [6]:
teeth = pd.DataFrame()
teeth['file_name'] = [i for i in os.listdir('../input/augmented-teeth/clear_transformed_dataset') if i.endswith('.jpg')]
teeth['file_name'] = teeth['file_name'].apply(lambda x: f'augmented-teeth/clear_transformed_dataset/{x}')

dop_teeth = pd.DataFrame()
dop_teeth['file_name'] = [i for i in os.listdir('../input/cutted-images-teeth/cutted_images_teeth') if i.endswith('.jpg')]
dop_teeth['file_name'] = dop_teeth['file_name'].apply(lambda x: f'cutted-images-teeth/cutted_images_teeth/{x}')

teeth = teeth.append(dop_teeth, ignore_index=True)
teeth['class'] = 'teeth'



caries = pd.DataFrame()
caries['file_name'] = [i for i in os.listdir('../input/cutted-images-teeth/cutted_images_caries') if i.endswith('.jpg')]
caries['file_name'] = caries['file_name'].apply(lambda x: f'cutted-images-teeth/cutted_images_caries/{x}')

dop_caries = pd.DataFrame()
dop_caries['file_name'] = [i for i in os.listdir('../input/augmented-teeth/caries_transformed_dataset') if i.endswith('.jpg')]
dop_caries['file_name'] = dop_caries['file_name'].apply(lambda x: f'augmented-teeth/caries_transformed_dataset/{x}')

caries = caries.append(dop_caries, ignore_index=True)
caries['class'] = 'caries'

train = teeth.append(caries, ignore_index=True).replace({'teeth':0, 'caries':1})

In [7]:
train

,file_name,class
0,augmented-teeth/clear_transformed_dataset/676_...,0
1,augmented-teeth/clear_transformed_dataset/940_...,0
2,augmented-teeth/clear_transformed_dataset/749_...,0
3,augmented-teeth/clear_transformed_dataset/87_4...,0
4,augmented-teeth/clear_transformed_dataset/826_...,0
...,...,...
8729,augmented-teeth/caries_transformed_dataset/112...,1
8730,augmented-teeth/caries_transformed_dataset/0_ ...,1
8731,augmented-teeth/caries_transformed_dataset/213...,1
8732,augmented-teeth/caries_transformed_dataset/621...,1


In [8]:
label2idx = {0:'teeth', 1:'caries'}
model = 'google/vit-base-patch16-384'
root_dir = '../input/'
batch_size = 8
num_labels = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
train, val, _, _ = train_test_split(train, train['class'], test_size=0.1)

In [10]:
class LandmarkDataset(Dataset):

    def __init__(self, meta, root_dir, transform=None):
        self.meta = meta
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_id = self.meta['file_name'].iloc[idx]
        img_name = self.root_dir + image_id
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image=np.asarray(image))['image']
            image = Image.fromarray(image)

        image = feature_extractor(images=image, return_tensors="pt")
            
        cls = self.meta['class'].iloc[idx]

        return image, cls

In [11]:
train_transform = A.Compose([
        A.HorizontalFlip(),
        A.GaussNoise(p=0.2),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.2, rotate_limit=30, p=0.75),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
            A.PiecewiseAffine(p=0.3),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.Sharpen(),
            A.Emboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    ])

transform = None

In [12]:
feature_extractor = ViTFeatureExtractor.from_pretrained(model)
feature_extractor.save_pretrained('feature_extractor')

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [13]:
train_dataset = LandmarkDataset(meta=train, root_dir=root_dir, transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

val_dataset = LandmarkDataset(meta=val, root_dir=root_dir, transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
train_dataloader.dataset.meta

,file_name,class
2163,augmented-teeth/clear_transformed_dataset/140_...,0
4173,cutted-images-teeth/cutted_images_teeth/1111_2...,0
387,augmented-teeth/clear_transformed_dataset/709_...,0
8201,cutted-images-teeth/cutted_images_caries/0_ (4...,1
2496,augmented-teeth/clear_transformed_dataset/516_...,0
...,...,...
6137,cutted-images-teeth/cutted_images_teeth/1188_2...,0
664,augmented-teeth/clear_transformed_dataset/662_...,0
7540,cutted-images-teeth/cutted_images_teeth/530_17...,0
7221,cutted-images-teeth/cutted_images_teeth/0_ (25...,0


In [15]:
model = ViTModel.from_pretrained(model)
model.to(device)

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-384 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-384 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )

In [16]:
EPOCHS = 10
lr = 0.00001

# Зададим оптимизатор
optimizer = optim.AdamW(model.parameters(), lr) # ЗДЕСЬ ВАШ КОД

criterion = losses.ArcFaceLoss(num_labels, 768).to(device)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS, eta_min=1e-9)

In [17]:
for epoch in range(EPOCHS):
    
    model.train()     
        
    train_loss = []
    for batch, targets in tqdm(train_dataloader, desc=f"Epoch: {epoch}"):
        optimizer.zero_grad()
        
        for i in batch:
            batch[i] = batch[i][:, 0].to(device)
        targets = targets.to(device)
        predictions = model(**batch).pooler_output
        
        loss = criterion(predictions, targets) 
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        
    print('Training loss:', np.mean(train_loss))
    
    model.eval()
        
    val_loss = []
    for batch, targets in tqdm(val_dataloader, desc=f"Epoch: {epoch}"):
        
        with torch.no_grad():
        
            for i in batch:
                batch[i] = batch[i][:, 0].to(device)
            targets = targets.to(device)
            predictions = model(**batch).pooler_output

            loss = criterion(predictions, targets) 

            val_loss.append(loss.item())
        
    print('Val loss:', np.mean(val_loss))
    model.save_pretrained('model_v2_30ep')

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0:   0%|          | 0/983 [00:00<?, ?it/s]

Training loss: 4.083833299879851


Epoch: 0:   0%|          | 0/110 [00:00<?, ?it/s]

Val loss: 2.003614803972993


...

Training loss: 1.131575223042947


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0>
Traceback (most recent call last):
Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0><function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0>  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0><function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0>Traceback (most recent call last):

Exception ignored in:   File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    

Epoch: 9:   0%|          | 0/110 [00:00<?, ?it/s]


Exception ignored in: 

    Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0>Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f9074c9f0e0>self._shutdown_workers()Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Traceback (most recent call last):
self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
      File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__

  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__


    self._shutdown_workers()  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
      File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320

Val loss: 0.6141277124300938


In [18]:
train_loss

[5.591072082519531,
 0.014769005589187145,
 1.3149384260177612,
 0.005213107913732529,
 0.0020270212553441525,
 1.8303933143615723,
 7.409972190856934,
 0.002632059156894684,
 0.005222938023507595,
 4.409846782684326,
 3.9835283756256104,
 0.001893252250738442,
 0.0007629742613062263,
 0.0010055771563202143,
 0.037058185786008835,
 0.0010982237290591002,
 0.0013182573020458221,
 4.136666297912598,
 0.0010781217133626342,
 3.8864715099334717,
 0.0020772479474544525,
 3.941514253616333,
 0.012798582203686237,
 0.0031506954692304134,
 4.193572998046875,
 0.011977093294262886,
 0.007923250086605549,
 0.0037737577222287655,
 0.0033830413594841957,
 0.019380759447813034,
 0.016564812511205673,
 3.8831727504730225,
 0.032707273960113525,
 0.007556392811238766,
 3.5999603271484375,
 0.00772241223603487,
 0.019881416112184525,
 3.7762081623077393,
 0.035376667976379395,
 0.06699054688215256,
 3.5780692100524902,
 0.03494613990187645,
 0.021313617005944252,
 0.09513312578201294,
 3.7152822017669

In [19]:
model

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )

In [20]:
model.eval()

res = []
base_embs = {}

for label in ['teeth', 'caries']:
    img_path = [i for i in os.listdir(f'../input/cutted-images-teeth/cutted_images_{label}') if i.endswith('.jpg')]

    for file in tqdm(img_path):
        image = Image.open(f'../input/cutted-images-teeth/cutted_images_{label}/{file}').convert('RGB')
        image = feature_extractor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            res.append(model(**image).pooler_output.detach().cpu().numpy())
            
    mean_emb = np.mean(np.asarray(res), axis=0)[0]
    base_embs.update({label:mean_emb})

  0%|          | 0/3982 [00:00<?, ?it/s]

  0%|          | 0/385 [00:00<?, ?it/s]

In [21]:
pd.DataFrame(base_embs).to_csv('base_file_teeth_v2_30ep.csv', index=False)